In [18]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from imblearn.combine import SMOTEENN

In [19]:
# load data
random_state=42
best_models = {}

df = pd.read_csv('../Data/Final_skin_cancer.csv')
df.drop('drink', axis=1, inplace=True)


In [20]:
# data.drop(['fitspatrick','region_hash_0','region_hash_1','region_hash_2','region_hash_3','region_hash_4','region_hash_5','region_hash_6','region_hash_7','region_hash_8','region_hash_9','changed','bleed','elevation','biopsed','diameter_2','diameter_1','itch','grew','hurt'], axis=1, inplace=True)

In [21]:
# data

In [22]:
from imblearn.under_sampling import ClusterCentroids, TomekLinks
from imblearn.over_sampling import RandomOverSampler

def splitting_data(df, sampling):
    X = df.drop(['diagnostic'], axis=1)
    y = df['diagnostic']

    if sampling == 'none':
        return X, y
    elif sampling == 'SMOTEENN':
        sampler = SMOTEENN(random_state=random_state)
    elif sampling == 'SMOTE':
        sampler = SMOTE(random_state=random_state)
    elif sampling == 'under':
        sampler = RandomUnderSampler(random_state=random_state)
    elif sampling == 'over':
        sampler = RandomOverSampler(random_state=random_state)
    elif sampling == 'cluster_centroids':
        sampler = ClusterCentroids(random_state=random_state)
    elif sampling == 'tomek_links':
        sampler = TomekLinks()

    X_resampled, y_resampled = sampler.fit_resample(X, y)
    return X_resampled, y_resampled


In [23]:

def training(X_train, y_train):

    HGB = HistGradientBoostingClassifier()
    # Fit the classifier to the data
    HGB.fit(X_train, y_train)
    return HGB

In [24]:
def best_model(modelName, accuracy, precision, recall, f1):
    best_models[modelName] = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [25]:
def predict(HGB, X_test ,y_test):
    # Predict the labels for the training data X
    y_pred = HGB.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    cr=classification_report(y_test, y_pred)
    print(cr)

In [26]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

def optimize_with_grid(X_train, y_train):

    # Initialize the LGBMClassifier
    HGB = HistGradientBoostingClassifier()
    # Define the parameter grid
    param_grid = {
        'max_iter': [100, 200, 300],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.1, 0.2],
        'min_samples_leaf': [20, 40, 60]
    }

    # Initialize GridSearchCV
    HGB_cv = GridSearchCV(HGB, param_grid, cv=5)

    # Fit the grid search to the data
    HGB_cv.fit(X_train, y_train)

    # Best parameters and best score
    best_params = HGB_cv.best_params_
    best_score = HGB_cv.best_score_
    best_estimator = HGB_cv.best_estimator_
    print(best_params)
    print(best_score)

    return best_estimator

# Original Dataset

In [27]:
# using function with no sampling
X, y= splitting_data(df, 'none')
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
# Scale the features using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [28]:
#check number of observations in each class in the set
print("Number of observations in each class in the training set:")
print(y.value_counts())

Number of observations in each class in the training set:
diagnostic
1    1494
0     211
Name: count, dtype: int64


In [29]:
HGB1 = training(X_train, y_train)
y_pred = predict(HGB1, X_test, y_test)

              precision    recall  f1-score   support

           0       0.84      0.73      0.78        37
           1       0.97      0.98      0.98       304

    accuracy                           0.96       341
   macro avg       0.91      0.86      0.88       341
weighted avg       0.95      0.96      0.95       341



In [30]:
best_HGB1 = optimize_with_grid(X_train, y_train)
prediction = predict('original_grid',best_HGB1, X_test, y_test)

{'learning_rate': 0.1, 'max_depth': 3, 'max_iter': 200, 'min_samples_leaf': 20}
0.942073367808662


TypeError: predict() takes 3 positional arguments but 4 were given

# SMOTE for over-sampling

In [ ]:
X,y = splitting_data(data, 'SMOTE')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
#check number of observations in each class in the set
print("Number of observations in each class in the training set:")
print(y.value_counts())

Number of observations in each class in the training set:
1    1013
0    1013
Name: Class, dtype: int64


In [ ]:
HGB2 =training(X_train, y_train)
y_pred = predict('SMOTE',HGB2, X_test, y_test)

              precision    recall  f1-score   support

           0       0.87      0.90      0.88       211
           1       0.88      0.86      0.87       195

    accuracy                           0.88       406
   macro avg       0.88      0.88      0.88       406
weighted avg       0.88      0.88      0.88       406



In [ ]:
best_HGB2 = optimize_with_grid(X_train, y_train)
prediction = predict('SMOTE_grid',best_HGB2, X_test, y_test)

{'learning_rate': 0.1, 'max_depth': 3, 'max_iter': 100, 'min_samples_leaf': 60}
0.8925925925925926
              precision    recall  f1-score   support

           0       0.86      0.91      0.88       211
           1       0.89      0.85      0.87       195

    accuracy                           0.88       406
   macro avg       0.88      0.88      0.88       406
weighted avg       0.88      0.88      0.88       406



# HGB using SMOTEENN sampling 

In [ ]:
X,y = splitting_data(data, 'SMOTEENN')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
#check number of observations in each class in the set
print("Number of observations in each class in the training set:")
print(y.value_counts())

Number of observations in each class in the training set:
1    156
0    137
Name: Class, dtype: int64


In [ ]:
HGB3 =training(X_train, y_train)
y_pred = predict('SMOTEENN',HGB3, X_test, y_test)

              precision    recall  f1-score   support

           0       0.85      0.71      0.77        24
           1       0.82      0.91      0.86        35

    accuracy                           0.83        59
   macro avg       0.84      0.81      0.82        59
weighted avg       0.83      0.83      0.83        59



In [ ]:
# from joblib import dump
# dump(HGB3,'../Models/HGB3_SMOTEENN.joblib')

['../Models/HGB3_SMOTEENN.joblib']

In [ ]:
best_HGB3 = optimize_with_grid(X_train, y_train)
prediction = predict('SMOTEENN_grid',best_HGB3, X_test, y_test)

{'learning_rate': 0.01, 'max_depth': 3, 'max_iter': 300, 'min_samples_leaf': 20}
0.8332099907493061
              precision    recall  f1-score   support

           0       0.88      0.88      0.88        24
           1       0.91      0.91      0.91        35

    accuracy                           0.90        59
   macro avg       0.89      0.89      0.89        59
weighted avg       0.90      0.90      0.90        59



# HGB on Random undersampling

In [ ]:
X,y = splitting_data(data, 'under')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
#check number of observations in each class in the set
print("Number of observations in each class in the training set:")
print(y.value_counts())

Number of observations in each class in the training set:
0    987
1    987
Name: Class, dtype: int64


In [ ]:
HGB4 =training(X_train, y_train)
y_pred = predict('under',HGB4, X_test, y_test)

              precision    recall  f1-score   support

           0       0.87      0.91      0.89       200
           1       0.90      0.86      0.88       195

    accuracy                           0.89       395
   macro avg       0.89      0.89      0.89       395
weighted avg       0.89      0.89      0.89       395



In [ ]:
best_HGB4 = optimize_with_grid(X_train, y_train)
prediction = predict('under_grid',best_HGB4, X_test, y_test)

{'learning_rate': 0.1, 'max_depth': 7, 'max_iter': 100, 'min_samples_leaf': 20}
0.8954992967651195
              precision    recall  f1-score   support

           0       0.86      0.89      0.87       200
           1       0.88      0.85      0.86       195

    accuracy                           0.87       395
   macro avg       0.87      0.87      0.87       395
weighted avg       0.87      0.87      0.87       395



# HGB on Random Oversampling

In [ ]:
X,y = splitting_data(data, 'over')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
#check number of observations in each class in the set
print("Number of observations in each class in the training set:")
print(y.value_counts())

Number of observations in each class in the training set:
1    1013
0    1013
Name: Class, dtype: int64


In [ ]:
HGB5 =training(X_train, y_train)
y_pred = predict('over',HGB5, X_test, y_test)

              precision    recall  f1-score   support

           0       0.89      0.91      0.90       211
           1       0.90      0.88      0.89       195

    accuracy                           0.89       406
   macro avg       0.89      0.89      0.89       406
weighted avg       0.89      0.89      0.89       406



In [ ]:
best_HGB5 = optimize_with_grid(X_train, y_train)
prediction = predict('over_grid',best_HGB5, X_test, y_test)

{'learning_rate': 0.1, 'max_depth': 3, 'max_iter': 100, 'min_samples_leaf': 20}
0.8962962962962964
              precision    recall  f1-score   support

           0       0.86      0.91      0.89       211
           1       0.90      0.85      0.87       195

    accuracy                           0.88       406
   macro avg       0.88      0.88      0.88       406
weighted avg       0.88      0.88      0.88       406



# HGB on Cluster Centroids

In [ ]:
X,y = splitting_data(data, 'cluster_centroids')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
#check number of observations in each class in the set
print("Number of observations in each class in the training set:")
print(y.value_counts())

Number of observations in each class in the training set:
0    987
1    987
Name: Class, dtype: int64


In [ ]:
HGB6 =training(X_train, y_train)
y_pred = predict('cluster_centroids',HGB6, X_test, y_test)

              precision    recall  f1-score   support

           0       0.86      0.90      0.88       200
           1       0.89      0.86      0.87       195

    accuracy                           0.88       395
   macro avg       0.88      0.88      0.88       395
weighted avg       0.88      0.88      0.88       395



In [ ]:
best_HGB6 = optimize_with_grid(X_train, y_train)
prediction = predict('cluster_centroids_grid',best_HGB6, X_test, y_test)

{'learning_rate': 0.2, 'max_depth': 3, 'max_iter': 100, 'min_samples_leaf': 40}
0.8992947558770343
              precision    recall  f1-score   support

           0       0.86      0.90      0.88       200
           1       0.89      0.85      0.87       195

    accuracy                           0.87       395
   macro avg       0.87      0.87      0.87       395
weighted avg       0.87      0.87      0.87       395



# HGB on Tomek Links

In [ ]:
X,y = splitting_data(data, 'tomek_links')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
#check number of observations in each class in the set
print("Number of observations in each class in the training set:")
print(y.value_counts())

Number of observations in each class in the training set:
1    987
0    694
Name: Class, dtype: int64


In [ ]:
HGB7 =training(X_train, y_train)
y_pred = predict('tomek_links',HGB7, X_test, y_test)

              precision    recall  f1-score   support

           0       0.86      0.87      0.87       139
           1       0.91      0.90      0.91       198

    accuracy                           0.89       337
   macro avg       0.89      0.89      0.89       337
weighted avg       0.89      0.89      0.89       337



In [ ]:
best_HGB7 = optimize_with_grid(X_train, y_train)
prediction = predict('tomek_links_grid',best_HGB7, X_test, y_test)

{'learning_rate': 0.1, 'max_depth': 3, 'max_iter': 100, 'min_samples_leaf': 20}
0.8988237252399711
              precision    recall  f1-score   support

           0       0.89      0.88      0.88       139
           1       0.92      0.92      0.92       198

    accuracy                           0.91       337
   macro avg       0.90      0.90      0.90       337
weighted avg       0.90      0.91      0.90       337



In [ ]:
best_model_df = pd.DataFrame.from_dict(best_models, orient='index')
best_model_df.sort_values(by='accuracy', ascending=False, inplace=True)
best_model_df

,accuracy,precision,recall,f1
original_grid,0.915000,0.915243,0.915000,0.915030
original,0.912500,0.912491,0.912500,0.912490
tomek_links_grid,0.905045,0.904899,0.905045,0.904939
SMOTEENN_grid,0.898305,0.898305,0.898305,0.898305
over,0.894089,0.894226,0.894089,0.894021
tomek_links,0.890208,0.890339,0.890208,0.890265
under,0.886076,0.886814,0.886076,0.885984
over_grid,0.879310,0.880174,0.879310,0.879092
SMOTE,0.876847,0.877034,0.876847,0.876748
SMOTE_grid,0.876847,0.877528,0.876847,0.876652
